In [1]:
from email.mime import image
import os
import zipfile
import requests

from pathlib import Path

data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'

if image_path.is_dir():
    print(f"{image_path} already exists")
else:
    print(f"Downloading images to {image_path}...")
    r = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    with open('pizza_steak.zip', 'wb') as f:
        f.write(r.content)
    print("Unzipping...")
    zip_ref = zipfile.ZipFile('pizza_steak.zip', 'r')
    zip_ref.extractall(data_path)
    zip_ref.close()
    print("Done")
    
    


Unzipping...
Done


In [2]:
train_dir = image_path / 'train'
test_dir = image_path / 'test'

test_dir, train_dir

(WindowsPath('data/pizza_steak_sushi/test'),
 WindowsPath('data/pizza_steak_sushi/train'))

In [3]:
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Resize, Compose

# Create a simple transform
data_transforms = transforms.Compose([
    Resize((64,64)),
    ToTensor()
])

# Use the ImageFolder to create dataset(s)
train_dataset = ImageFolder(root='data/train', 
                            transform=data_transforms,
                            target_transform=None) # Transform to perform on labels if needed

test_dataset = ImageFolder(root='data/test',
                        transform=data_transforms)

len(train_dataset), len(test_dataset)

(225, 75)

In [4]:
class_names = train_dataset.classes
print(class_names)

class_dict = train_dataset.class_to_idx
print(class_dict)

['pizza', 'steak', 'sushi']
{'pizza': 0, 'steak': 1, 'sushi': 2}


In [5]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, 
                            batch_size=1,
                            num_workers=1, # How many subprocesses to use for data loading
                            shuffle=True)

test_dataloader = DataLoader(test_dataset,
                            batch_size=1,
                            shuffle=False)

train_dataloader, train_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x241a998d6d0>,
 <torch.utils.data.dataloader.DataLoader at 0x241a998d6d0>)

In [6]:
img, label = next(iter(train_dataloader))

print(f"""
      Image shape: {img.shape}
      Label shape: {label.shape}
      """)



      Image shape: torch.Size([1, 3, 64, 64])
      Label shape: torch.Size([1])
      


In [7]:
# Making a TinyVGG Model

import torch
from torch import nn


class TinyVGG(nn.Module):
    """
    Creates the TinyVGG architecture.
    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
    """

    def __init__(self,
                input_shape: int = 3,
                hidden_units: int = 64,
                output_shape: int = 3) -> None:
        
        super().__init__()

        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*16*16,
                    out_features=output_shape))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.classifier(
                    self.conv_block_2(
                        self.conv_block_1(x)
                        )
                    )

In [8]:

import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Instantiate model
model_0 = TinyVGG(input_shape=3, hidden_units=64, output_shape=len(train_dataset.classes)).to(device)

model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=16384, out_features=3, bias=True)
  )
)

In [9]:
img_batch, label_batch = next(iter(train_dataloader))

img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single.to(device))

print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0189, -0.0017,  0.0222]], device='cuda:0')

Output prediction probabilities:
tensor([[0.3352, 0.3284, 0.3364]], device='cuda:0')

Output prediction label:
tensor([2], device='cuda:0')

Actual label:
2


In [10]:
from typing import Tuple


# Create a train and test_step function
import torch
def train_step(model: nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer, 
               device: torch.device) -> Tuple[float, float]:
    
    """
    Trains a PyTorch model for one epoch.
    
    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).
    
    Args:
        model: A PyTorch model instance.
        dataloader: A DataLoader instance for the model to be trained on.
        loss_fn: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        device: A target device to compute on (e.g. "cuda" or "cpu").
            If not specified, the device will be "cuda" if available.
    
    Returns:
        A Tuple of training loss and training accuracy metrics.
    
    """
    
    # Set model to training mode
    model.train()
    
    # Instantiate training loss and accuracy
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Forward pass
        y_pred = model(X)
        
        
        # Calculate loss and accumulate train loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        
        # Optimizer zero grad
        optimizer.zero_grad()

        # Backward pass
        loss.backward()
        
        # Optimizer step
        optimizer.step()
        
        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)
    
    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [11]:
def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:
    
    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval() 
  
  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0
  
  # Turn on inference context manager
  with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
                # Send data to target device
                X, y = X.to(device), y.to(device)
        
                # 1. Forward pass
                test_pred_logits = model(X)

                # 2. Calculate and accumulate loss
                loss = loss_fn(test_pred_logits, y)
                test_loss += loss.item()
                
                # Calculate and accumulate accuracy
                test_pred_labels = test_pred_logits.argmax(dim=1)
                test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
          
  # Adjust metrics to get average loss and accuracy per batch 
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

In [12]:
from typing import Dict, List

from tqdm.auto import tqdm

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]} 
    For example if training for epochs=2: 
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]} 
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }
  
  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)
      
      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

In [13]:
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.
  
  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)
  
  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

In [14]:
# Set random seeds
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 5

# Recreate an instance of TinyVGG
model_0 = TinyVGG(input_shape=3, # number of color channels (3 for RGB) 
                  hidden_units=10, 
                  output_shape=len(train_dataset.classes)).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Train model_0 
model_0_results = train(model=model_0, 
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS,
                        device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# Save the model
save_model(model=model_0,
           target_dir="models",
           model_name="05_going_modular_cell_mode_tinyvgg_model.pt")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1182 | train_acc: 0.2978 | test_loss: 1.0972 | test_acc: 0.4133
Epoch: 2 | train_loss: 1.1012 | train_acc: 0.3022 | test_loss: 1.0995 | test_acc: 0.2533
Epoch: 3 | train_loss: 1.0992 | train_acc: 0.3111 | test_loss: 1.0998 | test_acc: 0.3333
Epoch: 4 | train_loss: 1.0990 | train_acc: 0.3378 | test_loss: 1.1001 | test_acc: 0.3333
Epoch: 5 | train_loss: 1.0988 | train_acc: 0.3200 | test_loss: 1.1006 | test_acc: 0.3333
[INFO] Total training time: 21.686 seconds
[INFO] Saving model to: models\05_going_modular_cell_mode_tinyvgg_model.pt


### Create Datasets and Dataloaders in Script Mode

Use the Juypter Magic command to create a `.py` script

This save's a code cell's contents to a file using the Juypter magic `%%writefile filename`

In [15]:
# Create a directory going_modular

import os

# Check if the directory already exists
if os.path.isdir('going_modular'):
    print("going_modular directory already exists")
else:
    os.mkdir('going_modular')

going_modular directory already exists


In [13]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for image classification datasets.
"""
import os

from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = 2

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = NUM_WORKERS
):
    """
    Create dataloaders for training and testing datasets.
    
    Args:
        train_dir (str): The directory path of the training dataset.
        test_dir (str): The directory path of the testing dataset.
        transform (torchvision.transforms.Compose): The transformation to apply to the images.
        batch_size (int): The batch size for the dataloaders.
        num_workers (int, optional): The number of worker processes for data loading. Defaults to NUM_WORKERS.
    
    Returns:
        tuple: A tuple containing the training dataloader, testing dataloader, and class names.
    """
    train_dataset = ImageFolder(root=train_dir, 
                                transform=transform,
                                target_transform=None) # Transform to perform on labels if needed

    test_dataset = ImageFolder(root=test_dir,
                               transform=transform)
    
    train_dataloader = DataLoader(train_dataset, 
                                  batch_size=batch_size,
                                  num_workers=num_workers, # How many subprocesses to use for data loading
                                  shuffle=True,
                                  pin_memory=True) # If true, the data loader will copy Tensors into CUDA pinned memory before returning them

    test_dataloader = DataLoader(test_dataset,
                                 batch_size=batch_size,
                                 num_workers=num_workers, # How many subprocesses to use for data loading
                                 shuffle=False,
                                 pin_memory=True)

    class_names = train_dataset.classes
    
    return train_dataloader, test_dataloader, class_names
    
    

Overwriting going_modular/data_setup.py


In [20]:
train_dir = 'C:\\Users\\jakev\\Code\\Learn_Pytorch\\Practice\\data\\train'
test_dir = 'C:\\Users\\jakev\Code\\Learn_Pytorch\\Practice\\data\\test'

In [21]:
from going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=data_transforms,
                                                                               batch_size=32)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x24366615e90>,
 ['pizza', 'steak', 'sushi'])

In [22]:
%%writefile going_modular/model_setup.py
"""
Contains functionality for creating PyTorch computer vision models. 
"""

import torch
from torch import nn


class TinyVGG(nn.Module):
    """
    Creates the TinyVGG architecture.
    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
    """

    def __init__(self,
                input_shape: int = 3,
                hidden_units: int = 64,
                output_shape: int = 3) -> None:
        
        super().__init__()

        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*16*16,
                    out_features=output_shape))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.classifier(
                    self.conv_block_2(
                        self.conv_block_1(x)
                        )
                    )

Writing going_modular/model_setup.py


In [25]:
import torch

from going_modular import model_setup

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Instantiate model
model_1 = model_setup.TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names)).to(device)
model_1.state_dict()

OrderedDict([('conv_block_1.0.weight',
              tensor([[[[ 0.0392, -0.1832,  0.1811],
                        [ 0.0482,  0.0717, -0.1301],
                        [ 0.1239, -0.1029, -0.0437]],
              
                       [[-0.0912, -0.1740,  0.1270],
                        [-0.0854,  0.0818, -0.1743],
                        [-0.1581, -0.1337, -0.1644]],
              
                       [[-0.0010,  0.0632, -0.1180],
                        [-0.0927,  0.1662,  0.1698],
                        [-0.0702, -0.1762,  0.0872]]],
              
              
                      [[[ 0.1087, -0.0078, -0.1267],
                        [-0.1430,  0.0171,  0.0134],
                        [-0.0361,  0.0922,  0.1769]],
              
                       [[ 0.1547, -0.1376,  0.1208],
                        [-0.1004,  0.1757, -0.1643],
                        [-0.0065,  0.1217, -0.0438]],
              
                       [[ 0.0218, -0.1345, -0.0816],
                 

In [7]:
%%writefile going_modular/engine.py

"""
Contains functions for training and testing a PyTorch model. 
"""

import torch
import torch.nn as nn

from tqdm.auto import tqdm
from typing import Dict, List, Tuple


def train_step(model: nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer, 
               device: torch.device) -> Tuple[float, float]:
    
    """
    Trains a PyTorch model for one epoch.
    
    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).
    
    Args:
        model: A PyTorch model instance.
        dataloader: A DataLoader instance for the model to be trained on.
        loss_fn: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        device: A target device to compute on (e.g. "cuda" or "cpu").
            If not specified, the device will be "cuda" if available.
    
    Returns:
        A Tuple of training loss and training accuracy metrics.
    
    """
    
    # Set model to training mode
    model.train()
    
    # Instantiate training loss and accuracy
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Forward pass
        y_pred = model(X)
        
        
        # Calculate loss and accumulate train loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        
        # Optimizer zero grad
        optimizer.zero_grad()

        # Backward pass
        loss.backward()
        
        # Optimizer step
        optimizer.step()
        
        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)
    
    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc


def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:
    
    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval() 
  
  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0
  
  # Turn on inference context manager
  with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
                # Send data to target device
                X, y = X.to(device), y.to(device)
        
                # 1. Forward pass
                test_pred_logits = model(X)

                # 2. Calculate and accumulate loss
                loss = loss_fn(test_pred_logits, y)
                test_loss += loss.item()
                
                # Calculate and accumulate accuracy
                test_pred_labels = test_pred_logits.argmax(dim=1)
                test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
          
  # Adjust metrics to get average loss and accuracy per batch 
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc


from tqdm.auto import tqdm

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]} 
    For example if training for epochs=2: 
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]} 
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }
  
  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)
      
      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results


Overwriting going_modular/engine.py


In [2]:
%%writefile going_modular/utils.py
"""
Contains the functionality for saving a PyTorch model.
"""
import torch

from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.
  
  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)
  
  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


### Train, evaluate and save the model (script mode) -> train.py
Let's create a file called train.py to leverage all of our other code scripts to train a PyTorch Model
Essentially replicate the functionality of 04 notebook

In [14]:
%%writefile going_modular/train.py
"""
Contains the functionality for training a PyTorch image classification model.
"""

import os
import torch

from torchvision import transforms
from timeit import default_timer as timer

import data_setup, engine, model_setup, utils


# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "C:\\Users\\jakev\\Code\\Learn_Pytorch\\Practice\\data\\train"
test_dir = "C:\\Users\\jakev\\Code\\Learn_Pytorch\\Practice\\data\\test"

# Setup device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Create transforms
data_transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

# Create DataLoaders and get class names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=data_transforms,
                                                                               batch_size=BATCH_SIZE)

# Create model
model = model_setup.TinyVGG(input_shape=3, 
                            hidden_units=HIDDEN_UNITS, 
                            output_shape=len(class_names)).to(device)

# Set up loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Start timer
start_time = timer()

# Start training with engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             optimizer=optimizer,
             loss_fn=loss_fn,
             epochs=NUM_EPOCHS,
             device=device)

# End timer 
end_time = timer()
total_time = end_time - start_time
print(f"[INFO] Total training time: {total_time:.3f} seconds")

utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pt")

Overwriting going_modular/train.py


In [16]:
!python going_modular/train.py